In [1]:
import pandas as pd
import numpy as np
import requests
import csv

In [2]:
#pythia_url = "https://github.com/cms-sw/genproductions/blob/master/python/ThirteenTeV/Higgs/HH/ResonanceDecayFilter_example_HHTo4B_madgraph_pythia8_CP5_cff.py"
#pythia_RAW_url = pythia_url.replace('github.com','raw.githubusercontent.com').replace('/blob','')
#r = requests.get(pythia_RAW_url)
#fragment = r.text

#index         = fragment.find('from Configuration.Generator.Pythia8CommonSettings_cfi import')
#HH_4B_fragment = fragment[index:]


# Adding PSweight in Pythia fragment
# from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *
# pythia8PSweightsSettingsBlock,
# 'pythia8PSweightsSettings',

LHEproducer = """import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/mg242legacy/bin/MadGraph5_aMCatNLO/cards/production/pre2017/13TeV/exo_diboson/Spin-2/BulkGraviton_hh_narrow/BulkGraviton_hh_GF_HH_narrow_M900
"""

pythiaFragment = """from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     processParameters = cms.vstring(
                                                                                     '25:onMode = off',
                                                                                     '25:onIfMatch = 5 -5',
                                                                                     'ResonanceDecayFilter:filter = on'
                                                                                     ),
                                                     parameterSets = cms.vstring('pythia8CommonSettings',
                                                                                 'pythia8CP5Settings',
                                                                                 'pythia8PSweightsSettings',
                                                                                 'processParameters'
                                                                                 )
                                                     )
                         )
ProductionFilterSequence = cms.Sequence(generator)"""

In [3]:
print(LHEproducer+"\n"+pythiaFragment)

import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/mg242legacy/bin/MadGraph5_aMCatNLO/cards/production/pre2017/13TeV/exo_diboson/Spin-2/BulkGraviton_hh_narrow/BulkGraviton_hh_GF_HH_narrow_M900

from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),


In [6]:
masses16 = ["250", "280", "320", "700", "850", "1000", "1250", "1500", "1750", "2000", "2500", "3000"]
masses17 = ["400", "450", "500", "550", "600", "650", "800", "1000", "1250", "1500", "1750", "2000", "2500"]
masses18 = ["250", "260", "270", "280", "300", "320", "350", "400", "450", "500", "550", "600", "650", "700", "750", "800", "850", "900", "1000", "1250", "1500", "1750", "2000", "2500", "3000"]
events = [400000]*len(masses18)
with open('BulkGraviton_HH_4B_2018.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator', 'mcdbid','time','size'])

    for mass, events in zip(masses18, events):
        version="2.4.2"
        generators="Madgraph_" + version + "  Pythia8"
        gp = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/2017/13TeV/madgraph/V5_2.4.2/BulkGraviton_hh_GF_HH_narrow_M{0}/v1/BulkGraviton_hh_GF_HH_narrow_M{0}_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz".format(mass)
        dataset_name = "GluGluToBulkGravitonToHHTo4B_M-{0}_narrow_TuneCP5_13TeV-madgraph-pythia8".format(mass)
        final_fragment = LHEproducer.replace('__GRIDPACK__',gp) + '\n\n' + pythiaFragment
        note = "GluGlu BulkGraviton production m_BG = {0} GeV decaying into HH (m_H = 125 GeV) in the 4b final state".format(mass)
        mcdb_id = '0'
        time = '0.3'
        size = '120'
        csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
        